# Human-Segmentation


## Setup environment

In [16]:
!pip install iglovikov_helper_functions
!pip install people_segmentation  > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for jpeg4py: filename=jpeg4py-0.1.4-py3-none-any.whl size=8423 sha256=164afb447e1db3ed3f708dfc80330aa2029ca4ec4dda62d8b0e8a2e0cfb5a883
  Stored in directory: /root/.cache/pip/wheels/86/c3/0f/348e6cadb3a27435e833d21d91707d653fb159d69f2a867a36
Successfully built jpeg4py


In [17]:
%matplotlib inline
import numpy as np
import cv2
import os

from pylab import imshow
import torch
import albumentations as albu

from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image

from people_segmentation.pre_trained_models import create_model

## Setup model


In [18]:
transform = albu.Compose([albu.Normalize(p=1)], p=1)

In [19]:
model = create_model("Unet_2020-07-20")
model.eval();

Downloading: "https://github.com/ternaus/people_segmentation/releases/download/0.0.1/2020-09-23a.zip" to /root/.cache/torch/hub/checkpoints/2020-09-23a.zip
100%|██████████| 47.0M/47.0M [00:02<00:00, 17.6MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:665: UserWarning: Falling back to the old format < 1.6. This support will be deprecated in favor of default zipfile format introduced in 1.6. Please redo torch.save() to save it in the new zipfile format.
  warnings.warn('Falling back to the old format < 1.6. This support will be '


## Process video

In [70]:
%cd /content

/content


In [21]:
!gdown 1Vrh8ucwZ7QAdUicHspAW2_lV_dOekKS0

Downloading...
From: https://drive.google.com/uc?id=1Vrh8ucwZ7QAdUicHspAW2_lV_dOekKS0
To: /content/video.mp4
100% 1.02M/1.02M [00:00<00:00, 141MB/s]


In [82]:
!rm -rf frames_origin

In [83]:
import time

video_path = 'video.mp4'
output_folder = 'frames_origin'
os.makedirs(output_folder, exist_ok=True)

cap = cv2.VideoCapture(video_path)

frame_rate = 38  # Desired frame rate in frames per second
frame_interval = 1.0 / frame_rate

frame_count = 0
start_time = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    elapsed_time = current_time - start_time

    # Check if it's time to capture the next frame
    if elapsed_time >= frame_interval:
        frame_count += 1
        count = str(frame_count)
        if len(count) < 2:
          count = '0' + count
        frame_filename = os.path.join(output_folder, f'frame_{count}.jpg')
        cv2.imwrite(frame_filename, frame)
        start_time = current_time

cap.release()
cv2.destroyAllWindows()

## Create mask

In [63]:
origin_path = 'frames_origin/'

mask_path = 'frames_mask/'
os.makedirs(mask_path, exist_ok=True)

In [86]:
device = torch.device('cuda:0')

model.to(device)
with torch.no_grad():
  for dirpath, dirnames, filenames in os.walk(origin_path):
    if not dirnames:
      for file in filenames:
        image = load_rgb(dirpath + file)

        padded_image, pads = pad(image, factor=32, border=cv2.BORDER_CONSTANT)
        x = transform(image=padded_image)["image"]
        x = torch.unsqueeze(tensor_from_rgb_image(x), 0)
        x = x.to(device)

        prediction = model(x)[0][0]
        mask = (prediction > 0).cpu().numpy().astype(np.uint8) * 255
        mask = unpad(mask, pads)

        cv2.imwrite(mask_path + file, mask)

# Video-Inpainting

## Setup environment

In [2]:
# Install MMCV
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu101/torch1.5/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu101/torch1.5/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.4/605.4 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 31.2 MB/s eta 0:00:00
  Created wheel for mmcv-full: filename=mmcv_full-1.7.1-cp310-cp310-linux_x86_64.whl size=30392885 sha256=07ef89ccb5cf5c2582cf8baeccb2ad61b5bc03c26583c2f401ede721df11e57c
  Stored in directory: /root/.cache/pip/wheels/47/9a/65/470be18e21a8f2d085a024f0731508273543de0b5f79d9ddd4
Successfully built mmcv-full


In [25]:
!git clone https://github.com/KhaLee2307/video-inpainting.git

Cloning into 'video-inpainting'...
remote: Enumerating objects: 345, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 345 (delta 48), reused 32 (delta 32), pack-reused 268
Receiving objects: 100% (345/345), 36.75 MiB | 12.11 MiB/s, done.
Resolving deltas: 100% (53/53), done.


## Setup model

In [26]:
%cd video-inpainting/release_model
!gdown 1tNJMTJ2gmWdIXJoHVi5-H504uImUiJW9
!unzip E2FGVI_CVPR22_models.zip

/content/video-inpainting/release_model
Downloading...
From: https://drive.google.com/uc?id=1tNJMTJ2gmWdIXJoHVi5-H504uImUiJW9
To: /content/video-inpainting/release_model/E2FGVI_CVPR22_models.zip
100% 202M/202M [00:00<00:00, 225MB/s]
Archive:  E2FGVI_CVPR22_models.zip
  inflating: E2FGVI-CVPR22.pth       
  inflating: i3d_rgb_imagenet.pt     
/content/video-inpainting


## Inference


In [87]:
%cd video-inpainting
torch.cuda.empty_cache()

/content/video-inpainting


In [93]:
!python test.py --model e2fgvi --video /content/frames_origin --mask /content/frames_mask --ckpt release_model/E2FGVI-CVPR22.pth --savefps 10 --neighbor_stride 5

/usr/local/lib/python3.10/dist-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(
load pretrained SPyNet...
load checkpoint from http path: https://download.openmmlab.com/mmediting/restorers/basicvsr/spynet_20210409-c6c1bd09.pth
Loading model from: release_model/E2FGVI-CVPR22.pth
Loading videos and masks from: /content/frames_origin | INPUT MP4 format: False
Start test...
  0% 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.mesh